In [4]:
import os
import nltk
# you'll need to run nltk.download() and download all packages
# nltk.download()

In [5]:
from nltk.tokenize import sent_tokenize, word_tokenize

Sets from http://www.iaees.org/publications/journals/selforganizology/articles/2016-3(3)/algorithm-to-transform-natural-language-into-SQL-queries.pdf
Escape Word Set<br/>
Expression Mapping Set<br/>
Noun Set<br/>
Verb Set<br/>
Semantic Set<br/>
Variable Set<br/>
Relation Set<br/>
Attribute Set<br/>
Conjunction Set<br/>
This is all the sets for words and rules:

In [6]:
escape_words = ["a", "an", "the", "which", "is", "of", "with", "to", "for", "are", "and", "should", "be"]
escape_words.sort()

rules_on_top_of = []

rules_side_by_side = []

nouns = ["block1", "block2", "block3", "block4", "block5", "block6"]

verbs = ["on top of", "side by side"]

In [8]:
data = "Block6 should be on top of block4. Block3 needs to be side by side with block2."
sentences = sent_tokenize(data)
words = word_tokenize(sentences[0])
temp = []
for w in words:
    temp.append(w.lower())
words = temp[0:-1]
words

['block6', 'should', 'be', 'on', 'top', 'of', 'block4']